In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
from scipy.interpolate import interp1d
from scipy.ndimage import gaussian_filter
from sklearn.utils import check_random_state
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import svm, metrics
import datetime as dt

import wfdb

## Read Data

In [ ]:
rawData = []
label = [];

for i in range(45):
    index = '{:03}'.format(i)
    fileName = 'LabWalks/co' + index + '_base'
    if os.path.isfile(fileName + '.hea'):
        record = wfdb.rdrecord(fileName)
        data = record.p_signal
        rawData.append(data)
        label.append(0)
    
    index = '{:03}'.format(i)
    fileName = 'LabWalks/fl' + index + '_base'
    if os.path.isfile(fileName + '.hea'):
        record = wfdb.rdrecord(fileName)
        data = record.p_signal
        rawData.append(data)
        label.append(1)

index = 3
print(np.shape(rawData[index]))
plt.plot(rawData[index][:,0:3])
a = gaussian_filter(rawData[index], sigma = 10)
#plt.plot(a)

## Check One Data

In [ ]:
from scipy.signal import find_peaks

for index in range(1):
    one = rawData[index]
    Fs = 100;
    for i in range(3):
        timeSeq = one[:, i] - np.mean(one[:, i])
        N = len(timeSeq)
        frqSeq = np.fft.fft(timeSeq)
        frqSeqShift = np.fft.fftshift(frqSeq)
        #print(type(frqSeq))
        frqSeqHalf = frqSeq[:N//2+1]
        psd = 1/(Fs*N) * abs(frqSeqHalf)**2
        psd[2:] = psd[2:]*2
        freq = np.linspace(0,Fs/2,N//2+1)
        plt.figure()
        plt.plot(freq, abs(psd), lw=2)
        peaks, _ = find_peaks(psd)
        plt.plot(freq[peaks], psd[peaks], "x")



In [ ]:
NumOfSamples = 1000

one = rawData[5]

accNorm = np.linalg.norm(one[:,0:3],axis=1)
gyrNorm = np.linalg.norm(one[:,3:],axis=1)
refData = gaussian_filter(accNorm, sigma = 10)
#check
checkData = []
checkTime = []
checkDataVar = []
#results
oneTime = []
oneIMUdata = [[] for i in range(6)]

index = 0
while index + 1 < len(one):
    startIndex = index
    #print(startIndex)
    while index + 1 < len(one) and refData[index] < max(refData[index-1], refData[index+1]):
        index = index + 1
    index = index + 1
    endIndex = index  
    #print(endIndex)
    if endIndex - startIndex > 20:
        checkTime.append(endIndex - startIndex)

    if endIndex < len(one) - 1 and endIndex - startIndex > 0.8*np.mean(checkTime) and endIndex - startIndex < 1.2*np.mean(checkTime):            
        usedData = accNorm[startIndex: endIndex]
        f = interp1d(range(len(usedData)), usedData, kind='cubic')
        x = np.linspace(0, len(usedData)-1, num=NumOfSamples, endpoint=False)
        processedData = f(x)
        checkDataVar.append(np.var(processedData))
        if(np.var(processedData) > 0.5 * np.mean(checkDataVar)):
            oneTime.append(endIndex - startIndex)
            checkData.append(processedData)
            for i in range(6):
                usedData = one[startIndex: endIndex, i]
                f = interp1d(range(len(usedData)), usedData, kind='cubic')
                x = np.linspace(0, len(usedData)-1, num=NumOfSamples, endpoint=False)
                oneIMUdata[i].append(f(x))    

checkData = np.array(checkData)
print(len(checkData))

    
checkData = np.array(checkData)
print(np.mean(checkTime))
print(len(checkData))
print(np.std(checkData, axis=1))

print(np.shape(np.array(oneIMUdata)))

plt.plot(np.std(np.array(checkData),axis=0)/np.mean(np.array(checkData),axis=0))
plt.show()

## Period Segmentation

In [ ]:
NumOfSamples = 1000

IMUdata = []
allTime = []

for one in rawData:

    accNorm = np.linalg.norm(one[:,0:3],axis=1)
    gyrNorm = np.linalg.norm(one[:,3:],axis=1)
    refData = gaussian_filter(accNorm, sigma = 10)
    #check
    checkData = []
    checkTime = []
    checkDataVar = []
    #results
    oneTime = []
    oneIMUdata = [[] for i in range(6)]
    
    index = 0
    while index + 1 < len(one):
        startIndex = index
        #print(startIndex)
        while index + 1 < len(one) and refData[index] < max(refData[index-1], refData[index+1]):
            index = index + 1
        index = index + 1
        endIndex = index  
        #print(endIndex)
        if endIndex - startIndex > 20:
            checkTime.append(endIndex - startIndex)

        if endIndex < len(one) - 1 and endIndex - startIndex > 0.8*np.mean(checkTime) and endIndex - startIndex < 1.2*np.mean(checkTime):            
            usedData = accNorm[startIndex: endIndex]
            f = interp1d(range(len(usedData)), usedData, kind='cubic')
            x = np.linspace(0, len(usedData)-1, num=NumOfSamples, endpoint=False)
            processedData = f(x)
            checkDataVar.append(np.var(processedData))
            if(np.var(processedData) > 0.2 * np.mean(checkDataVar)):
                oneTime.append(endIndex - startIndex)
                checkData.append(processedData)
                for i in range(6):
                    usedData = one[startIndex: endIndex, i]
                    f = interp1d(range(len(usedData)), usedData, kind='cubic')
                    x = np.linspace(0, len(usedData)-1, num=NumOfSamples, endpoint=False)
                    oneIMUdata[i].append(f(x))    
                
    IMUdata.append(oneIMUdata)
    allTime.append(oneTime)
    
print(len(IMUdata), len(allTime))

## Extract Feature

In [ ]:
from scipy.signal import find_peaks
from scipy.signal import peak_widths


def psdFeatures(one):    
    re = [];
    Fs = 100;
    for i in range(3):
        timeSeq = one[:, i] - np.mean(one[:, i])
        N = len(timeSeq)
        frqSeq = np.fft.fft(timeSeq)
        frqSeqShift = np.fft.fftshift(frqSeq)
        #print(type(frqSeq))
        frqSeqHalf = frqSeq[:N//2+1]
        psd = 1/(Fs*N) * abs(frqSeqHalf)**2
        psd[2:] = psd[2:]*2
        freq = np.linspace(0,Fs/2,N//2+1)
        #plt.figure(i)
        #plt.plot(freq, abs(psd), lw=2)
        peaks, _ = find_peaks(psd)
        peakWidth = peak_widths(psd, peaks, rel_height=0.5)
        maxPeakIndex = np.argmax(psd[peaks])
        re.append(psd[maxPeakIndex])
        re.append(peakWidth[0][maxPeakIndex])
        #print(peakWidth[0][maxPeakIndex]/np.amax(psd))
        #print(peakWidth[0][maxPeakIndex]/(N//2+1)/psd[maxPeakIndex])
        re.append(peakWidth[0][maxPeakIndex]/np.amax(psd))
        re.append(freq[maxPeakIndex])
    return re
        
        
        

In [ ]:
X = []
y = label

for i in range(len(IMUdata)):
    oneX = []
    #time related feature
    oneX.append(len(allTime[i]))
    oneX.append(np.mean(allTime[i]))
    oneX.append(np.std(allTime[i]))
    oneX.append(np.std(allTime[i])/(np.mean(allTime[i])))
    
    oneX = oneX + psdFeatures(rawData[i])
    
    #acc related feature
    for j in range(6):
        data = IMUdata[i][j]
        oneX.append(np.mean(np.abs(np.mean(data, axis=0))))
        oneX.append(np.std(np.abs(np.mean(data, axis=0))))
        oneX.append(np.mean(np.std(data, axis=0)))
        oneX.append(np.std(np.std(data, axis=0)))
        oneX.append(np.mean(np.std(data, axis=0)/np.abs(np.mean(data, axis=0))))
        oneX.append(np.std(np.std(data, axis=0)/np.abs(np.mean(data, axis=0))))

    X.append(oneX)

scaler = StandardScaler()
Xs = scaler.fit_transform(X)
print(np.shape(np.array(X)))
print(np.mean(X, axis=0))

## Visualize Data

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2).fit(Xs)
pca_2d = pca.transform(Xs)


import pylab as pl
for i in range(0, pca_2d.shape[0]):
    if y[i] == 0:
        c1 = pl.scatter(pca_2d[i,0],pca_2d[i,1],c='r',    marker='+')
    elif y[i] == 1:
        c2 = pl.scatter(pca_2d[i,0],pca_2d[i,1],c='g',    marker='o')

        
pl.legend([c1, c2], ['Nonfaller', 'Faller'])
pl.title('Training dataset with 2 classes')
pl.show()

pca.explained_variance_ratio_

In [ ]:
def plot_embedding(X, y, title=None):
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)     
    plt.figure()
    ax = plt.subplot(111)
    for i in range(X.shape[0]):
        plt.text(X[i, 0], X[i, 1], str(y[i]),
                 color=plt.cm.Set1(y[i] / 10.),
                 fontdict={'weight': 'bold', 'size': 9})
    plt.xticks([]), plt.yticks([])
    if title is not None:
        plt.title(title)
        
from sklearn import manifold
#降维
tsne = manifold.TSNE(n_components=2, init='pca')

X_tsne = tsne.fit_transform(X)
#绘图
plot_embedding(X_tsne,y*5)
plt.show()

## Train and Test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3)

from sklearn.decomposition import PCA
pca = PCA(n_components=2).fit(X_train)

#X_tf_pca = pca.transform(X_train)
#X_tsf_pca = pca.transform(X_test)

X_tf_pca = X_train
X_tsf_pca = X_test


#X_tf_pca = X_train
#X_tsf_pca = X_test

param_C = 1
param_gamma = 0.00002

classifier = svm.SVC(C=param_C,gamma=param_gamma,tol=0.0001)
#classifier = svm.LinearSVC(C=param_C)

#We learn the digits on train part
start_time = dt.datetime.now()
print('Start learning at {}'.format(str(start_time)))
classifier.fit(X_tf_pca, y_train)
#classifier.fit(X_pca, y)

end_time = dt.datetime.now() 
print('Stop learning {}'.format(str(end_time)))
elapsed_time= end_time - start_time
print('Elapsed learning {}'.format(str(elapsed_time)))

expected = y_train
predicted = classifier.predict(X_tf_pca)
print(expected)
print(predicted)

cm = metrics.confusion_matrix(expected, predicted)
print("Confusion matrix:\n%s" % cm)
print("Accuracy={}".format(metrics.accuracy_score(expected, predicted)))


expected = y_test
predicted = classifier.predict(X_tsf_pca)
print(expected)
print(predicted)

cm = metrics.confusion_matrix(expected, predicted)
print("Confusion matrix:\n%s" % cm)
print("Accuracy={}".format(metrics.accuracy_score(expected, predicted)))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1)

from sklearn.model_selection import cross_val_score

param_C = 1
param_gamma = 0.00002

classifier = svm.SVC(C=param_C,gamma=param_gamma,tol=0.001)
scores = cross_val_score(classifier, X_train, y_train, cv=10)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
#We learn the digits on train part

In [ ]:
from sklearn.model_selection import cross_validate



recall = []
precision = []
f1 = []



pca = PCA(n_components=40).fit(X_train)
#X_tf_pca = pca.transform(X_train)
#X_tsf_pca = pca.transform(X_test)

X_tf_pca = X_train
X_tsf_pca = X_test

for i in range(100):
    X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.0001)
    
    
    
    pca = PCA(n_components=51).fit(X_train)
    X_tf_pca = pca.transform(X_train)
    X_tsf_pca = pca.transform(X_test)

    #X_tf_pca = X_train
    #X_tsf_pca = X_test
    
    scoring = ['precision', 'recall', 'f1']

    param_C = 0.5
    param_gamma = 0.00001

    classifier = svm.SVC(C=param_C,gamma=param_gamma,tol=0.001)

    scores = cross_validate(classifier, X_tf_pca, y_train, scoring=scoring,
                             cv=5)

    recall.append(np.mean(scores['test_recall']))
    precision.append(np.mean(scores['test_precision']))    
    f1.append(np.mean(scores['test_f1']))
print(np.mean(recall))
print(np.mean(precision))                      
print(np.mean(f1))                      



In [ ]:
from sklearn.ensemble import RandomForestClassifier

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2)

pca = PCA(n_components=40).fit(X_train)
X_tf_pca = pca.transform(X_train)
X_tsf_pca = pca.transform(X_test)


rnd_clf = RandomForestClassifier(n_estimators=2, max_leaf_nodes=10000, n_jobs=-1)
rnd_clf.fit(X_tf_pca, y_train)

#expected = y_test
#predicted = rnd_clf.predict(X_tsf_pca)

expected = y_train
predicted = rnd_clf.predict(X_tf_pca)

print(expected)
print(predicted)

cm = metrics.confusion_matrix(expected, predicted)
print("Confusion matrix:\n%s" % cm)

print("Accuracy={}".format(metrics.accuracy_score(expected, predicted)))


expected = y_test
predicted = rnd_clf.predict(X_tsf_pca)

print(expected)
print(predicted)

cm = metrics.confusion_matrix(expected, predicted)
print("Confusion matrix:\n%s" % cm)

print("Accuracy={}".format(metrics.accuracy_score(expected, predicted)))